In [ ]:
import pandas as pd
import time
from sklearn.feature_selection import mutual_info_classif
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [2]:
df = pd.read_parquet('/home/samuelw/Projetos/tedas/data/raw/mimic_iv_ext_onto_sepse_rotulado.parquet')
df.head()

,subject_id,hadm_id,stay_id,charttime,fc,pas,pad,pam,fr,spo,...,leu,pla,ph,uri,hem,ida,pes,alt,sexo,tem_sepse
0,10000032,29079034,39553978,2180-07-23 14:11:00,NaN,84.0,48.0,56.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
1,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,NaN,NaN,NaN,24.0,NaN,...,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
2,10000032,29079034,39553978,2180-07-23 14:13:00,NaN,NaN,NaN,NaN,NaN,98.0,...,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
3,10000032,29079034,39553978,2180-07-23 14:30:00,93.0,95.0,59.0,67.0,21.0,97.0,...,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
4,10000032,29079034,39553978,2180-07-23 15:00:00,94.0,88.0,56.0,64.0,23.0,97.0,...,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0


### Vendo quais as colunas do dataset

In [3]:
colunas = list(df.columns)
print(colunas)

['subject_id', 'hadm_id', 'stay_id', 'charttime', 'fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'cre', 'lac', 'leu', 'pla', 'ph', 'uri', 'hem', 'ida', 'pes', 'alt', 'sexo', 'tem_sepse']


In [ ]:
df.shape()

#### Identificando e eliminando duplicatas

In [4]:
# Início da medição de tempo
start_time = time.time()

# Detecta as duplicatas
duplicatas = df.duplicated()

# Exibe as duplicatas encontradas
print("Linhas duplicadas:")
print(df[duplicatas])

# Remove as duplicatas (mantém a primeira ocorrência)
df = df.drop_duplicates()

# Fim da medição de tempo
end_time = time.time()

# Exibe tempo de execução
print("-" * 100)
elapsed_time = end_time - start_time
print(f"Tempo de execução: {elapsed_time:.2f} segundos")
print("-" * 100)


Linhas duplicadas:
Empty DataFrame
Columns: [subject_id, hadm_id, stay_id, charttime, fc, pas, pad, pam, fr, spo, tem, cre, lac, leu, pla, ph, uri, hem, ida, pes, alt, sexo, tem_sepse]
Index: []

[0 rows x 23 columns]
----------------------------------------------------------------------------------------------------
Tempo de execução: 7.72 segundos
----------------------------------------------------------------------------------------------------


In [5]:
# ------------------------------------------------------
# Função de avaliação das características selecionadas
# Utiliza a média da informação mútua entre X e y
# ------------------------------------------------------
def avaliar_caracteristicas(indices_selecionados, X, y):
    if len(indices_selecionados) == 0:
        return 0
    X_selecionado = X[:, indices_selecionados]
    pontuacao = mutual_info_classif(X_selecionado, y).mean()
    return pontuacao

# ------------------------------------------------------
# Algoritmo ACO (Ant Colony Optimization) para seleção de características
# ------------------------------------------------------
def selecao_por_colonia_de_formigas(
    X, y, n_formigas=5, n_iteracoes=5,
    alfa=1, beta=1, taxa_evaporacao=0.5,
    max_caracteristicas_por_formiga=10
):
    n_caracteristicas = X.shape[1]

    # Vetor de feromônio inicial (uniforme)
    feromonio = np.ones(n_caracteristicas)

    # Armazena a melhor solução encontrada
    melhor_indice_caracteristicas = []
    melhor_pontuacao = -np.inf

    for iteracao in range(n_iteracoes):
        solucoes_iteracao = []
        pontuacoes_iteracao = []

        print(f"Iteração {iteracao + 1}/{n_iteracoes}")

        for formiga in range(n_formigas):
            # Calcula as probabilidades com base no feromônio
            probabilidades = feromonio ** alfa
            probabilidades /= probabilidades.sum()

            # Sorteia número aleatório de características até o máximo permitido
            tamanho_conjunto = np.random.randint(1, min(max_caracteristicas_por_formiga, n_caracteristicas) + 1)

            # Seleciona características com base nas probabilidades
            indices_selecionados = np.random.choice(
                range(n_caracteristicas),
                size=tamanho_conjunto,
                replace=False,
                p=probabilidades
            )

            # Avalia o subconjunto escolhido
            pontuacao = avaliar_caracteristicas(indices_selecionados, X, y)
            solucoes_iteracao.append(indices_selecionados)
            pontuacoes_iteracao.append(pontuacao)

            # Atualiza melhor solução global
            if pontuacao > melhor_pontuacao:
                melhor_pontuacao = pontuacao
                melhor_indice_caracteristicas = indices_selecionados

        # Atualização do feromônio com evaporação
        feromonio *= (1 - taxa_evaporacao)

        # Reforço positivo baseado na qualidade das soluções
        for solucao, pontuacao in zip(solucoes_iteracao, pontuacoes_iteracao):
            for indice in solucao:
                feromonio[indice] += pontuacao

    return melhor_indice_caracteristicas, melhor_pontuacao

# ------------------------------------------------------
# PREPARAÇÃO DOS DADOS
# ------------------------------------------------------

# Define os nomes das colunas de entrada (features) e da variável alvo (target)
colunas_entrada = ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'cre', 'lac', 'leu', 'pla', 'ph', 'uri', 'hem', 'ida']
coluna_alvo = 'tem_sepse'

# Seleciona apenas as colunas de interesse e força dados numéricos
dados_entrada = df[colunas_entrada].copy()
dados_entrada = dados_entrada.apply(pd.to_numeric, errors='coerce')

# Preenche valores ausentes com a média de cada coluna
dados_entrada.fillna(dados_entrada.mean(), inplace=True)

# Converte para arrays NumPy
X = dados_entrada.values.astype(float)
y = df[coluna_alvo].values

# Codifica y se for categórico
if y.dtype == 'object' or str(y.dtype).startswith("category"):
    y = LabelEncoder().fit_transform(y)

# ------------------------------------------------------
# EXECUÇÃO DO ACO
# ------------------------------------------------------

melhores_indices, melhor_pontuacao = selecao_por_colonia_de_formigas(X, y)

# Recupera os nomes das melhores características
melhores_nomes = [colunas_entrada[i] for i in melhores_indices]

# Mostra os resultados finais
print("Melhores características selecionadas:", melhores_nomes)
print("Melhor pontuação (média da informação mútua):", melhor_pontuacao)

Iteração 1/5


KeyboardInterrupt: 

In [7]:
# ------------------------------------------------------
# Função de avaliação das características selecionadas
# Utiliza a SOMA da informação mútua entre X e y
# ------------------------------------------------------
def avaliar_caracteristicas(indices_selecionados, X, y):
    if len(indices_selecionados) == 0:
        return 0
    X_selecionado = X[:, indices_selecionados]
    pontuacao = mutual_info_classif(X_selecionado, y).sum()
    return pontuacao

# ------------------------------------------------------
# Algoritmo ACO (Ant Colony Optimization) para seleção de características
# ------------------------------------------------------
def selecao_por_colonia_de_formigas(
    X, y,
    n_formigas=10,
    n_iteracoes=4,
    alfa=2,
    beta=1,  # beta não está sendo usado pois não há heurística, mas deixado por padrão
    taxa_evaporacao=0.3,
    min_caracteristicas_por_formiga=5,
    max_caracteristicas_por_formiga=10
):
    n_caracteristicas = X.shape[1]
    feromonio = np.ones(n_caracteristicas)

    melhor_indice_caracteristicas = []
    melhor_pontuacao = -np.inf

    for iteracao in range(n_iteracoes):
        solucoes_iteracao = []
        pontuacoes_iteracao = []

        print(f"Iteração {iteracao + 1}/{n_iteracoes}")

        for formiga in range(n_formigas):
            # Calcula as probabilidades com base no feromônio
            probabilidades = feromonio ** alfa
            probabilidades /= probabilidades.sum()

            # Define número de características no subconjunto (mínimo forçado)
            tamanho_conjunto = np.random.randint(
                min_caracteristicas_por_formiga,
                min(max_caracteristicas_por_formiga, n_caracteristicas) + 1
            )

            # Seleciona características com base nas probabilidades
            indices_selecionados = np.random.choice(
                range(n_caracteristicas),
                size=tamanho_conjunto,
                replace=False,
                p=probabilidades
            )

            # Avalia o subconjunto escolhido
            pontuacao = avaliar_caracteristicas(indices_selecionados, X, y)
            solucoes_iteracao.append(indices_selecionados)
            pontuacoes_iteracao.append(pontuacao)

            # Atualiza melhor solução global
            if pontuacao > melhor_pontuacao:
                melhor_pontuacao = pontuacao
                melhor_indice_caracteristicas = indices_selecionados

        # Evaporação do feromônio
        feromonio *= (1 - taxa_evaporacao)

        # Reforço positivo com base nas pontuações das soluções
        for solucao, pontuacao in zip(solucoes_iteracao, pontuacoes_iteracao):
            for indice in solucao:
                feromonio[indice] += pontuacao

    return melhor_indice_caracteristicas, melhor_pontuacao

# ------------------------------------------------------
# PREPARAÇÃO DOS DADOS
# ------------------------------------------------------

# Define os nomes das colunas de entrada (features) e da variável alvo (target)
colunas_entrada = ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'cre', 'lac', 'leu', 'pla', 'ph', 'uri', 'hem', 'ida']
coluna_alvo = 'tem_sepse'

# Seleciona e prepara os dados
dados_entrada = df[colunas_entrada].copy()
dados_entrada = dados_entrada.apply(pd.to_numeric, errors='coerce')
dados_entrada.fillna(dados_entrada.mean(), inplace=True)

# Converte para arrays NumPy
X = dados_entrada.values.astype(float)
y = df[coluna_alvo].values

# Codifica y se for categórico
if y.dtype == 'object' or str(y.dtype).startswith("category"):
    y = LabelEncoder().fit_transform(y)

# ------------------------------------------------------
# EXECUÇÃO DO ACO
# ------------------------------------------------------

melhores_indices, melhor_pontuacao = selecao_por_colonia_de_formigas(X, y)

# Recupera os nomes das melhores características
melhores_nomes = [colunas_entrada[i] for i in melhores_indices]

# Mostra os resultados finais
print("Melhores características selecionadas:", melhores_nomes)
print("Melhor pontuação (soma da informação mútua):", melhor_pontuacao)

Iteração 1/4
Iteração 2/4
Iteração 3/4
Iteração 4/4
Melhores características selecionadas: ['lac', 'cre', 'pas', 'pam', 'spo', 'tem', 'hem', 'pad', 'fr']
Melhor pontuação (soma da informação mútua): 1.0821580610284995


### Usando o beta na euristica


In [6]:
def selecao_por_colonia_de_formigas(
    X, y,
    n_formigas=10,
    n_iteracoes=8,
    alfa=1.0,
    beta=1.0,
    taxa_evaporacao=0.3,
    min_caracteristicas_por_formiga=5,
    max_caracteristicas_por_formiga=10
):
    n_caracteristicas = X.shape[1]

    # Inicializa vetor de feromônio
    feromonio = np.ones(n_caracteristicas)

    # Calcula a heurística (informação mútua de cada feature com o target)
    heuristica = mutual_info_classif(X, y)
    heuristica += 1e-6  # evita problemas com zero

    melhor_indice_caracteristicas = []
    melhor_pontuacao = -np.inf

    for iteracao in range(n_iteracoes):
        solucoes_iteracao = []
        pontuacoes_iteracao = []

        print(f"Iteração {iteracao + 1}/{n_iteracoes}")

        for formiga in range(n_formigas):
            # Cálculo das probabilidades combinando feromônio e heurística
            probabilidades = (feromonio ** alfa) * (heuristica ** beta)
            probabilidades /= probabilidades.sum()

            # Número de características a serem escolhidas pela formiga
            tamanho_conjunto = np.random.randint(
                min_caracteristicas_por_formiga,
                min(max_caracteristicas_por_formiga, n_caracteristicas) + 1
            )

            # Seleciona características com base nas probabilidades
            indices_selecionados = np.random.choice(
                range(n_caracteristicas),
                size=tamanho_conjunto,
                replace=False,
                p=probabilidades
            )

            # Avalia a solução da formiga
            pontuacao = avaliar_caracteristicas(indices_selecionados, X, y)
            solucoes_iteracao.append(indices_selecionados)
            pontuacoes_iteracao.append(pontuacao)

            # Atualiza a melhor solução encontrada até agora
            if pontuacao > melhor_pontuacao:
                melhor_pontuacao = pontuacao
                melhor_indice_caracteristicas = indices_selecionados

        # Evaporação do feromônio
        feromonio *= (1 - taxa_evaporacao)

        # Reforço positivo com base na qualidade das soluções
        for solucao, pontuacao in zip(solucoes_iteracao, pontuacoes_iteracao):
            for indice in solucao:
                feromonio[indice] += pontuacao

    return melhor_indice_caracteristicas, melhor_pontuacao

# ------------------------------------------------------
# PREPARAÇÃO DOS DADOS
# ------------------------------------------------------

# Define os nomes das colunas de entrada e da variável alvo
colunas_entrada = ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'cre',
                   'lac', 'leu', 'pla', 'ph', 'uri', 'hem', 'ida']
coluna_alvo = 'tem_sepse'

# Pré-processamento
dados_entrada = df[colunas_entrada].copy()
dados_entrada = dados_entrada.apply(pd.to_numeric, errors='coerce')
dados_entrada.fillna(dados_entrada.mean(), inplace=True)

X = dados_entrada.values.astype(float)
y = df[coluna_alvo].values

# Codifica y se for categórico
if y.dtype == 'object' or str(y.dtype).startswith("category"):
    y = LabelEncoder().fit_transform(y)

# ------------------------------------------------------
# EXECUÇÃO DO ACO COM BETA
# ------------------------------------------------------

melhores_indices, melhor_pontuacao = selecao_por_colonia_de_formigas(X, y)

melhores_nomes = [colunas_entrada[i] for i in melhores_indices]

# Resultados finais
print("Melhores características selecionadas:", melhores_nomes)
print("Melhor pontuação (soma da informação mútua):", melhor_pontuacao)

Iteração 1/8


NameError: name 'avaliar_caracteristicas' is not defined

### Utilizando RF

In [ ]:
# ------------------------------------------------------
# FUNÇÃO DE AVALIAÇÃO DAS CARACTERÍSTICAS
# ------------------------------------------------------
def avaliar_caracteristicas(indices, X, y):
    modelo = RandomForestClassifier(n_estimators=50, random_state=42)
    X_selecionado = X[:, indices]
    
    # Usa validação cruzada com 5 folds e média da acurácia
    scores = cross_val_score(modelo, X_selecionado, y, cv=5, scoring='accuracy')
    return scores.mean()

# ------------------------------------------------------
# FUNÇÃO ACO (Colônia de Formigas)
# ------------------------------------------------------
def selecao_por_colonia_de_formigas(
    X, y,
    n_formigas=10,
    n_iteracoes=4,
    alfa=1.5,
    beta=2.0,
    taxa_evaporacao=0.3,
    min_caracteristicas_por_formiga=5,
    max_caracteristicas_por_formiga=10
):
    n_caracteristicas = X.shape[1]
    feromonio = np.ones(n_caracteristicas)
    heuristica = mutual_info_classif(X, y)
    heuristica += 1e-6  # evita zeros

    melhor_indice_caracteristicas = []
    melhor_pontuacao = -np.inf

    for iteracao in range(n_iteracoes):
        print(f"Iteração {iteracao + 1}/{n_iteracoes}")
        solucoes_iteracao = []
        pontuacoes_iteracao = []

        for formiga in range(n_formigas):
            probabilidades = (feromonio ** alfa) * (heuristica ** beta)
            probabilidades /= probabilidades.sum()

            tamanho_conjunto = np.random.randint(
                min_caracteristicas_por_formiga,
                min(max_caracteristicas_por_formiga, n_caracteristicas) + 1
            )

            indices_selecionados = np.random.choice(
                range(n_caracteristicas),
                size=tamanho_conjunto,
                replace=False,
                p=probabilidades
            )

            pontuacao = avaliar_caracteristicas(indices_selecionados, X, y)
            solucoes_iteracao.append(indices_selecionados)
            pontuacoes_iteracao.append(pontuacao)

            if pontuacao > melhor_pontuacao:
                melhor_pontuacao = pontuacao
                melhor_indice_caracteristicas = indices_selecionados

        feromonio *= (1 - taxa_evaporacao)

        for solucao, pontuacao in zip(solucoes_iteracao, pontuacoes_iteracao):
            for indice in solucao:
                feromonio[indice] += pontuacao

    return melhor_indice_caracteristicas, melhor_pontuacao

# ------------------------------------------------------
# PREPARAÇÃO DOS DADOS
# ------------------------------------------------------

# Define os nomes das colunas de entrada e da variável alvo
colunas_entrada = ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'cre',
                   'lac', 'leu', 'pla', 'ph', 'uri', 'hem', 'ida']
coluna_alvo = 'tem_sepse'

# Pré-processamento
dados_entrada = df[colunas_entrada].copy()
dados_entrada = dados_entrada.apply(pd.to_numeric, errors='coerce')
dados_entrada.fillna(dados_entrada.mean(), inplace=True)

X = dados_entrada.values.astype(float)
y = df[coluna_alvo].values

# Codifica y se for categórico
if y.dtype == 'object' or str(y.dtype).startswith("category"):
    y = LabelEncoder().fit_transform(y)

# ------------------------------------------------------
# EXECUÇÃO DO ACO
# ------------------------------------------------------

melhores_indices, melhor_pontuacao = selecao_por_colonia_de_formigas(X, y)

melhores_nomes = [colunas_entrada[i] for i in melhores_indices]

# Resultados finais
print("Melhores características selecionadas:", melhores_nomes)
print("Melhor pontuação (acurácia média da validação cruzada):", melhor_pontuacao)

Iteração 1/4
Iteração 2/4
Iteração 3/4
